In [1]:
pip install geopandas pandas openpyxl pyproj shapely fiona

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 59.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import geopandas as gpd
import unicodedata as ud
from pathlib import Path

In [4]:
pop= pd.read_csv("/content/Lagos_pop.csv")
pop.head()

,State code,State,LGA code,LGA,Population Data Source,Female,Male,Total
0,LA,Lagos,25001,Agege,WorldPop and National Population Commission of...,"86,899","100,004","186,904"
1,LA,Lagos,25002,Ajeromi/Ifelodun,WorldPop and National Population Commission of...,"98,551","117,187","215,738"
2,LA,Lagos,25003,Alimosho,WorldPop and National Population Commission of...,"966,996","1,032,911","1,999,908"
3,LA,Lagos,25004,Amuwo Odofin,WorldPop and National Population Commission of...,"191,041","218,416","409,458"
4,LA,Lagos,25005,Apapa,WorldPop and National Population Commission of...,"57,809","72,438","130,247"


In [13]:
pop=pop[["LGA","Female","Male","Total"]]
pop

,LGA,Female,Male,Total
0,Agege,"86,899","100,004","186,904"
1,Ajeromi/Ifelodun,"98,551","117,187","215,738"
2,Alimosho,"966,996","1,032,911","1,999,908"
3,Amuwo Odofin,"191,041","218,416","409,458"
4,Apapa,"57,809","72,438","130,247"
5,Badagry,"285,174","295,384","580,557"
6,Epe,"118,514","121,597","240,110"
7,Eti Osa,"357,938","467,153","825,091"
8,Ibeju Lekki,"220,356","236,153","456,509"
9,Ifako/Ijaye,"192,501","211,526","404,027"


In [17]:
gdf = gpd.read_file("/content/GRID3_NGA_-_Operational_LGA_Boundaries.geojson")
lagos_gdf=gdf[gdf['statename']=='Lagos']
lagos_gdf=lagos_gdf[["lganame","geometry"]]
lagos_gdf

,lganame,geometry
249,Lagos Mainland,"POLYGON ((3.38516 6.47546, 3.38505 6.47532, 3...."
264,Lagos Island,"POLYGON ((3.4077 6.44936, 3.40736 6.44734, 3.4..."
306,Apapa,"POLYGON ((3.40284 6.43644, 3.40296 6.43419, 3...."
388,Mushin,"POLYGON ((3.33194 6.50373, 3.33076 6.50252, 3...."
410,Surulere,"POLYGON ((3.36363 6.4891, 3.36421 6.48751, 3.3..."
413,Shomolu,"POLYGON ((3.37456 6.52792, 3.37455 6.52787, 3...."
438,Agege,"POLYGON ((3.33307 6.60205, 3.33313 6.60191, 3...."
453,Oshodi/Isolo,"POLYGON ((3.3237 6.5061, 3.3228 6.50327, 3.322..."
699,Ajeromi/Ifelodun,"POLYGON ((3.35047 6.45278, 3.35105 6.45267, 3...."
701,Epe,"POLYGON ((4.05956 6.62407, 4.05756 6.62076, 4...."


In [20]:
# --- clean names for matching ---
lagos_gdf["lganame_clean"] = lagos_gdf["lganame"].str.strip().str.lower()
pop["LGA_clean"] = pop["LGA"].str.strip().str.lower()

# --- clean population numbers (remove commas, convert to int) ---
for col in ["Female", "Male", "Total"]:
    pop[col] = pop[col].astype(str).str.replace(",", "").astype(int)

# --- merge ---
merged = lagos_gdf.merge(
    pop,
    left_on="lganame_clean",
    right_on="LGA_clean",
    how="left"
)

# --- drop helper columns if you want ---
merged = merged.drop(columns=["lganame_clean", "LGA","LGA_clean"])


In [21]:
merged

,lganame,geometry,Female,Male,Total
0,Lagos Mainland,"POLYGON ((3.38516 6.47546, 3.38505 6.47532, 3....",116839,128254,245092
1,Lagos Island,"POLYGON ((3.4077 6.44936, 3.40736 6.44734, 3.4...",35137,38448,73585
2,Apapa,"POLYGON ((3.40284 6.43644, 3.40296 6.43419, 3....",57809,72438,130247
3,Mushin,"POLYGON ((3.33194 6.50373, 3.33076 6.50252, 3....",86816,94963,181779
4,Surulere,"POLYGON ((3.36363 6.4891, 3.36421 6.48751, 3.3...",109730,121232,230962
5,Shomolu,"POLYGON ((3.37456 6.52792, 3.37455 6.52787, 3....",81129,88268,169397
6,Agege,"POLYGON ((3.33307 6.60205, 3.33313 6.60191, 3....",86899,100004,186904
7,Oshodi/Isolo,"POLYGON ((3.3237 6.5061, 3.3228 6.50327, 3.322...",273789,302600,576389
8,Ajeromi/Ifelodun,"POLYGON ((3.35047 6.45278, 3.35105 6.45267, 3....",98551,117187,215738
9,Epe,"POLYGON ((4.05956 6.62407, 4.05756 6.62076, 4....",118514,121597,240110


In [24]:
# Reproject to UTM 31N (EPSG:32631) for accurate area calculations (Lagos falls within this zone)
merged = merged.to_crs(32631)

# Compute area in km²
merged['area_km2'] = merged.geometry.area / 1_000_000

# Compute population density
merged['density'] = merged['Total'] / merged['area_km2']

In [25]:
merged

,lganame,geometry,Female,Male,Total,area_km2,density
0,Lagos Mainland,"POLYGON ((542587.263 715779.044, 542575.516 71...",116839,128254,245092,20.165793,12153.848956
1,Lagos Island,"POLYGON ((545082.209 712896.627, 545044.558 71...",35137,38448,73585,5.044426,14587.387051
2,Apapa,"POLYGON ((544545.922 711467.437, 544559.407 71...",57809,72438,130247,37.371761,3485.171638
3,Mushin,"POLYGON ((536701.134 718900.558, 536570.583 71...",86816,94963,181779,16.994198,10696.532987
4,Surulere,"POLYGON ((540205.214 717285.144, 540269.867 71...",109730,121232,230962,19.997578,11549.498634
5,Shomolu,"POLYGON ((541410.933 721578.314, 541409.926 72...",81129,88268,169397,10.288772,16464.258380
6,Agege,"POLYGON ((536818.019 729769.984, 536824.889 72...",86899,100004,186904,12.240372,15269.470394
7,Oshodi/Isolo,"POLYGON ((535789.088 719162.328, 535690.557 71...",273789,302600,576389,53.998908,10674.086261
8,Ajeromi/Ifelodun,"POLYGON ((538753.246 713269.292, 538818.016 71...",98551,117187,215738,12.481240,17284.981584
9,Epe,"POLYGON ((617127.397 732316.558, 616906.389 73...",118514,121597,240110,1314.304268,182.689812


In [ ]:
# --- save final GeoJSON for Leaflet ---
merged.to_file("epsg32621.geojson", driver="GeoJSON")

In [ ]:
# Load your file (UTM CRS)
gdf = gpd.read_file("epsg32621.geojson")

# Reproject to WGS84 (EPSG:4326)
gdf = gdf.to_crs(epsg=4326)

# Save new GeoJSON
gdf.to_file("lagos_population.geojson", driver="GeoJSON")